In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# 간단하게 처리하는 방법
# from tensorflow.keras.datasets import imdb

# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [2]:
from tensorflow.keras.datasets import imdb
(X_train, y_train), (X_test, y_test) = imdb.load_data()
X_train

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1

In [5]:
import pandas as pd
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split


imdb_data = pd.read_csv("/content/drive/My Drive/NLP/data/IMDB Dataset.csv")
# pos, neg 값을 숫자로 변환
imdb_data['sentiment'] = imdb_data['sentiment'].replace('positive', 1)
# imdb_data['sentiment'].replace('positive', 1, inplace=True)
imdb_data['sentiment'] = imdb_data['sentiment'].replace('negative', 0)
# 정규표현식을 써서, 단어가 아니면 공백으로
imdb_data['review'] = imdb_data['review'].str.replace("[^\w]|br", " ")
# 혹시 공백이 있으면 제거
imdb_data['review'] = imdb_data['review'].replace("", np.nan)
imdb_data['sentiment'] = imdb_data['sentiment'].replace("", np.nan)
# null array 없애는 함수
imdb_data = imdb_data.dropna(how='any')

print("# preprocessing done")

review_train, review_test, y_train, y_test = train_test_split(imdb_data['review'], imdb_data['sentiment'], test_size=0.25, shuffle=False, random_state=23)

print("# split done")

stopwords = ['a', 'an']

X_train = []
for stc in review_train:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = stc.split()
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print("# tokenization done")

# 왜 트레인셋만?.. 원하신다면 처음에 test/train 스플릿하기 전에, 전처리하셔서 fit하셔도
# 왜 5000?.. 유의미한 단어 갯수를 생각해보자! 빈도수 1개다 -> 버리자, 빈도수 2개다 -> 버리자
tokenizer = Tokenizer(5000)
# train 을 기준으로 단어마다의 인덱스를 부여
tokenizer.fit_on_texts(X_train)

# 부여된 정수 인덱스로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print("# int_encoding done")

# preprocessing done
# split done
# tokenization done
# int_encoding done


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
print(X_train[1])
print(y_train[1])

[106, 387, 118, 357, 1, 1364, 3016, 5, 51, 51, 158, 54, 2198, 1550, 2, 409, 2, 527, 277, 3, 1847, 4, 1, 444, 408, 1, 150, 24, 566, 67, 2206, 494, 4098, 22, 60, 44, 192, 29, 1, 17, 23, 44, 29, 1, 2316, 176, 3408, 95, 19, 48, 371, 61, 1, 795, 32, 1, 1848, 4, 1798, 22, 60, 5, 6, 67, 273, 1, 147, 17, 6, 5, 412, 2, 2441, 408, 106, 4391, 357, 42, 27, 3, 1, 79, 1146, 11, 3, 201, 2, 25, 113, 1, 1847, 63, 267, 348, 15, 1, 118, 179, 1, 1030, 3, 1, 2791, 59, 249, 72, 356, 1, 2210, 960, 3198, 1249, 1202, 89, 4982, 6, 304, 21, 256, 1864, 2, 256, 4320, 572, 15, 1, 134, 3643, 2, 2, 1, 712, 572, 3, 64, 1047, 15, 11, 171, 2348, 24, 2010, 67, 218]
1


In [7]:
# 문장 길이를 맞춰준다
# 임의로 맞추는게 아니고, 데이터셋을 보면서 최대 문장의 길이가 얼마인지 확인하시고 거기에 맞춰서
# imdb일때는 500, 네이버일때는 50으로 맞췄었습니다!
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [8]:
print(X_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [9]:
# 레이어들을 쌓을 모델을 생성
model = Sequential()
# 단어를 임베딩하는데, 5000개 (imdb) 혹은 20000개 (네이버) 의 단어를 120차원으로 내보내겠다
# 1인자 = 내가 넣을 단어의 갯수 (총 인덱스의 갯수), 2인자 = 출력할 차원 (덴스 벡터의 차원), 3인자 = 문장의 길이
model.add(Embedding(5000, 120))
# RNN - simpleRNN / LSTM
model.add(LSTM(120))
# 긍정/부정을 판단하니까 이진 분류 -> sigmoid 함수 사용
model.add(Dense(1, activation='sigmoid'))

In [10]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [11]:
# 긍정/부정을 판단하니까 손실함수는 이진 교차 엔트로피, 최적화는 adam, 평가 기준은 acc (출력할때 뜬다)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[early_stop, model_check])

Epoch 1/10
586/586 [==============================] - 567s 964ms/step - loss: 0.5190 - acc: 0.7391 - val_loss: 0.3098 - val_acc: 0.8714

Epoch 00001: val_acc improved from -inf to 0.87144, saving model to the_best.h5
Epoch 2/10
586/586 [==============================] - 560s 956ms/step - loss: 0.2598 - acc: 0.8963 - val_loss: 0.2761 - val_acc: 0.8850

Epoch 00002: val_acc improved from 0.87144 to 0.88504, saving model to the_best.h5
Epoch 3/10
586/586 [==============================] - 563s 961ms/step - loss: 0.2159 - acc: 0.9160 - val_loss: 0.2991 - val_acc: 0.8886

Epoch 00003: val_acc improved from 0.88504 to 0.88856, saving model to the_best.h5
Epoch 4/10
586/586 [==============================] - 568s 970ms/step - loss: 0.1724 - acc: 0.9351 - val_loss: 0.2930 - val_acc: 0.8905

Epoch 00004: val_acc improved from 0.88856 to 0.89048, saving model to the_best.h5
Epoch 5/10
586/586 [==============================] - 576s 983ms/step - loss: 0.1484 - acc: 0.9449 - val_loss: 0.3442 - val

In [13]:
# 정확도 측정
# 출력하면 [loss, acc]
print(model.evaluate(X_test, y_test))
# X 값은 전처리, 토큰화, 정수인코딩이 된 상태의 문장이어야
# print(model.predict(X))

391/391 [==============================] - 44s 113ms/step - loss: 0.3735 - acc: 0.8612
[0.37350350618362427, 0.8611999750137329]


In [ ]:
!pip install konlpy

In [ ]:
########## 한글 전처리부분입니다 ########

import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split

naver_data = pd.read_table("./drive/My Drive/ratings.txt", encoding='utf-8')
# 단어 아니면 삭제
naver_data['document'] = naver_data['document'].str.replace("[^\w]", " ")
# 혹시 공백이 있으면 null array로
naver_data['document'] = naver_data['document'].replace('', np.nan)
naver_data['label'] = naver_data['label'].replace('', np.nan)
# null array 모두 제거 (공백인 열 모두 제거)
naver_data = naver_data.dropna(how='any')

print("# preproecssing done")

# test/train 스플릿하고
review_train, review_test, y_train, y_test = train_test_split(naver_data['document'], naver_data['label'], test_size=0.25, shuffle=False, random_state=1004)

print('# split done')

# stopwords 지정
stopwords = ['를', '을', '이', '가', '으로', '로', '에', '에서']

# 토큰화 진행
X_train = []
for stc in review_train:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)

X_test = []
for stc in review_test:
    token = []
    words = Okt().morphs(stc, stem=True)
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)

print('# tokenization done')

# X_train 단어들을 토대로 정수 인덱스 설정, 전체 단어 갯수 설정
# 유의미한 단어? 빈도수가 1~2개인 단어 버려도 큰 영향을 끼치지 않을것 -> count함수써서 빈도수 낮은 것들을 버리고, 남은 단어의 갯수들
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(X_train)

# 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print('# int_encoding done')